In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import random
import sklearn
import collections
from sklearn.model_selection import train_test_split
import json
import pylab
import tensorflow as tf
import tensorflow.math as tfmath
import tensorflow.keras as keras
from scipy.optimize import curve_fit
from tensorflow.keras import layers, Model
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from sklearn.metrics import roc_curve, auc
import sklearn.metrics as sk

2024-02-06 22:59:12.009842: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-06 22:59:12.126109: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
f=h5py.File('/eos/home-w/wsherman/AD_Work/n_tuples/40MHZ_data/background_for_training.h5','r')
Dataset=np.array(f["Particles"])

#for i, batch in enumerate(Dataset):
#  pt_sum=0
#  for j, particle in enumerate(Dataset[i,:,:]):
#    if particle[3]!=0:
#      pt_sum+=particle[0]
#  for j, particle in enumerate(Dataset[i,:,:]):
#    particle[0]=particle[0]/pt_sum
    
    
Data_Train=Dataset[0:2000000,:,0:3]
Data_Test=Dataset[2000001:3600000,:,0:3]
Data_Validate=Dataset[3600001:4000000,:,0:3]

Data_Train_Flat=np.reshape(Data_Train,(-1,57))
Data_Val_Flat=np.reshape(Data_Validate,(-1,57))

In [3]:
class Sampling(keras.layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


def make_encoder(input_dim,h_dim_1,h_dim_2,latent_dim):
    inputs=keras.Input(shape=(input_dim))
    #x=layers.BatchNormalization()(inputs)
    x=layers.Dense(h_dim_1, activation='relu')(inputs)
    x=layers.Dense(h_dim_2, activation='relu')(x)
    z_mean=layers.Dense(latent_dim, name='z_mean')(x)
    z_logvar=layers.Dense(latent_dim, name='z_log_var')(x)
    z=Sampling()([z_mean,z_logvar])
    encoder=keras.Model(inputs,[z_mean,z_logvar,z],name='encoder')
    return encoder




def make_decoder(input_dim,h_dim_1,h_dim_2,latent_dim):
    inputs=keras.Input(shape=(latent_dim))
    x=layers.Dense(h_dim_2, activation='relu')(inputs)
    x=layers.Dense(h_dim_1, activation='relu')(x)
    z=layers.Dense(input_dim)(x)
    decoder=keras.Model(inputs,z,name='decoder')
    return decoder

class VAE_Model(keras.Model):
    def __init__(self,encoder,decoder,**kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        self.beta=1

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]
    
    def set_beta(self,beta):
        self.beta=beta
    
    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            #making a masked loss function
            mask = K.cast(K.not_equal(data, 0), K.floatx())
            
            reconstruction_loss = tf.reduce_mean(tf.reduce_sum(keras.losses.mse(mask*data, mask*reconstruction)))
            reconstruction_loss *=(1-self.beta)

            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(kl_loss)
            kl_loss *=self.beta


            total_loss = reconstruction_loss + kl_loss
            
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reco_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def test_step(self, data):
        z_mean, z_log_var, z = self.encoder(data)
        
        reconstruction = self.decoder(z)
        mask = K.cast(K.not_equal(data, 0), K.floatx())
        reconstruction_loss = tf.reduce_mean(tf.reduce_sum(keras.losses.mse(mask*data, mask*reconstruction)))
        
        reconstruction_loss*=(1-self.beta)

        kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
        
        #KL loss changed abck to sum as in paper
        kl_loss = tf.reduce_sum(kl_loss)
        
        kl_loss *=self.beta
        
        total_loss = reconstruction_loss + kl_loss
        return {
            "loss": total_loss,
            "reco_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }

    def call(self, data):
        z_mean,z_log_var,x = self.encoder(data)
        reconstruction = self.decoder(x)
        return {
            "z_mean": z_mean,
            "z_log_var": z_log_var,
            "reconstruction": reconstruction
        }

        
    

In [4]:
#Here is a normalized model with (1-beta)rl beta*Kl loss
beta=0.83
vae_enc=make_encoder(57,32,16,3)
vae_dec=make_decoder(57,32,16,3)
vae_40MHZ=VAE_Model(vae_enc,vae_dec)
vae_40MHZ.set_beta(beta)
opt=keras.optimizers.Adam(learning_rate=0.001)
vae_40MHZ.compile(optimizer=opt)


STOP_PATIENCE = 8
LR_PATIENCE = 4

early_stopping = EarlyStopping(patience=STOP_PATIENCE, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=LR_PATIENCE, verbose=1)

callbacks = [early_stopping, reduce_lr]

NUM_EPOCHS = 100
BATCH_SIZE = 100

history = vae_40MHZ.fit(x=Data_Train_Flat, validation_split=0.1,epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks,shuffle=True)
vae_40MHZ.save_weights(filepath='/eos/home-w/wsherman/AD_Work/ML_git_repo/AD_trigger_training/trained_models/Different_VAE_Models/non_normed_new_beta_{}_v4_no_batchnorm/'.format(beta),save_format='tf')

Epoch 1/100
18000/18000 [==============================] - 51s 3ms/step - loss: 113.0161 - reco_loss: 95.8629 - kl_loss: 7.2736 - val_loss: 833.1437 - val_reco_loss: 8.3764 - val_kl_loss: 824.7673 - lr: 0.0010
Epoch 2/100
18000/18000 [==============================] - 46s 3ms/step - loss: 20.6478 - reco_loss: 17.5558 - kl_loss: 3.0718 - val_loss: 777.8638 - val_reco_loss: 7.7402 - val_kl_loss: 770.1237 - lr: 0.0010
Epoch 3/100
18000/18000 [==============================] - 46s 3ms/step - loss: 17.8315 - reco_loss: 15.1073 - kl_loss: 2.7481 - val_loss: 736.0100 - val_reco_loss: 7.2232 - val_kl_loss: 728.7869 - lr: 0.0010
Epoch 4/100
18000/18000 [==============================] - 45s 3ms/step - loss: 29.2283 - reco_loss: 14.4081 - kl_loss: 41.0303 - val_loss: 900.8517 - val_reco_loss: 6.1680 - val_kl_loss: 894.6838 - lr: 0.0010
Epoch 5/100
18000/18000 [==============================] - 44s 2ms/step - loss: 16.1860 - reco_loss: 13.3615 - kl_loss: 2.7319 - val_loss: 765.0691 - val_reco_los

In [5]:
a=Data_Train_Flat[0:100]
b=vae_40MHZ.encoder(a)
print(b)

[<tf.Tensor: shape=(100, 3), dtype=float32, numpy=
array([[-1.8108056e+00, -6.8010747e-02, -1.1608523e+00],
       [ 2.2932932e+00, -1.1156259e+00, -7.9223716e-01],
       [ 1.1637506e+00,  1.0667250e+00, -3.4212172e-01],
       [ 1.4196062e+00,  7.4135202e-01, -1.4708246e+00],
       [ 1.0880392e+00, -5.0630510e-01,  2.4788332e-01],
       [ 1.7308694e-01,  6.1555749e-01, -6.1812007e-01],
       [-7.6606154e-02,  1.0604949e+00, -6.4806461e-02],
       [-1.6179707e+00,  2.3087865e-01,  3.9841056e-02],
       [-3.1769425e-01,  4.6532989e-02,  2.3227453e-01],
       [-1.3215661e+00,  3.3368200e-01, -2.5617993e-01],
       [ 3.1449717e-01,  1.0733414e-01, -8.4114802e-01],
       [-1.3019562e-01,  3.4888810e-01, -1.2383441e+00],
       [ 4.9400014e-01,  4.1662371e-01,  3.5039926e-01],
       [-1.7167914e-01, -2.1303892e-01, -1.7050529e-01],
       [ 6.6929072e-01, -4.9087915e-01, -1.1899825e+00],
       [-1.7296348e+00,  1.3355010e+00,  1.4060739e+00],
       [-1.5893346e-01, -9.1810483e-0